In [ ]:
import requests
import pandas as pd

In [ ]:
def get_catalogs():
    r = requests.get('https://byui.kuali.co/api/v1/catalog/public/catalogs')
    r.raise_for_status()
    return r.json()

In [ ]:
def get_courses(cid):
    url = 'https://byui.kuali.co/api/v1/catalog/courses/{cid}'.format(cid=cid)
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

In [ ]:
def get_programs(cid):
    url = 'https://byui.kuali.co/api/v1/catalog/programs/{cid}'.format(cid=cid)
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

### Catalogs

In [ ]:
catalogs = []
for catalog in get_catalogs():
    catalogs.append({
        'id': catalog.get('_id'), 
        'startDate': catalog.get('startDate'),
        'endDate': catalog.get('endDate'),
        'title': catalog.get('title'),
    })
    
pd.DataFrame(catalogs).to_csv('catalogs.csv', index=False)

### Programs

In [ ]:
programs = []
for catalog in catalogs:
    for program in get_programs(catalog['id']):
        try:
            programs.append({
                'id': program.get('id'),
                'pid': program.get('pid'),
                'code': program.get('code'),
                'title': program.get('title'),
                'department': program.get('department').get('name'),
                'programType': program.get('programType').get('name'),
                'dateStart': program.get('dateStart'),
                'catalog': catalog.get('title'),
                'catalog_id': catalog.get('id'),
            })
        except:
            print('Didn\'t work for program in catalog {}'.format(catalog['id']))
            print(program)

pd.DataFrame(programs).to_csv('programs.csv', index=False)

### Courses

In [ ]:
courses = []
for catalog in catalogs:
    for course in get_courses(catalog['id']):
        try:
            courses.append({
                'id': course.get('id'),
                'pid': course.get('pid'),
                'code': course.get('__catalogCourseId'),
                'title': course.get('title'),
                'subject': course.get('subjectCode').get('description'),
                'subject_code': course.get('subjectCode').get('name'),
                'dateStart': course.get('dateStart'),
                'catalog': catalog.get('title'),
                'catalog_id': catalog.get('id'),
            })
        except:
            print('Didn\'t work for course in catalog {}'.format(catalog['id']))
            print(course)

pd.DataFrame(courses).to_csv('courses.csv', index=False)